### Transform CSV file of UberonIDs & Labels (with particular formatting) to JSON for SlicerMorph.

In [1]:
#import required pacakges

import json
import pandas as pd
import numpy as np

#### Read in CSV and rename `Slicer Label` column for ease of referencing

In [2]:
df = pd.read_csv("terminology.csv")

In [3]:
df.sample(10)

,Region,UberonID,UberonLabel,SlicerLabel,Paired,R,G,B,Notes
48,Forelimb,100048,Forelimb,Forelimb,Y,20,103,94,All long bones + girdle
63,Hindlimb,100063,Lower Leg,Lower Leg,Y,87,197,211,"Tibia, fibula and everything distal"
79,Spine,100079,Lumbar 3,Lumbar 3,NaN,58,149,252,NaN
60,Hindlimb,100060,Ischium,Ischium,Y,255,249,87,NaN
67,Hindlimb,100067,Patella,Patella,Y,101,56,199,NaN
72,Spine,100072,Cervical 3,Cervical 3,NaN,204,41,150,NaN
19,Head,100019,Vomer,Vomer,Y,219,46,46,NaN
40,Dentition,100040,Maxillary Incisor 3,Maxillary Incisor 3,Y,70,192,255,NaN
22,Dentition,100022,Maxillary Molar 1,Maxillary Molar 1,Y,0,179,92,NaN
69,Spine,100069,Sacrum,Sacrum,Y,41,95,204,NaN


Look at sample `Region`

In [4]:
df.loc[df.Region == "Forelimb"]

,Region,UberonID,UberonLabel,SlicerLabel,Paired,R,G,B,Notes
48,Forelimb,100048,Forelimb,Forelimb,Y,20,103,94,All long bones + girdle
49,Forelimb,100049,Pectoral Girdle,Pectoral Girdle,Y,26,52,27,"scapula, clavicles, humerus (can be incomplete)"
50,Forelimb,100050,Lower Arm,Lower Arm,Y,55,139,77,radius + ulna + everything distal
51,Forelimb,100051,Scapula,Scapula,Y,129,212,139,NaN
52,Forelimb,100052,Clavicle,Clavicle,Y,190,250,215,NaN
53,Forelimb,100053,Humerus,Humerus,Y,255,255,255,NaN
54,Forelimb,100054,Radius,Radius,Y,224,189,144,NaN
55,Forelimb,100055,Ulna,Ulna,Y,211,218,115,NaN


In [5]:
list(df.loc[df.Region == "Forelimb"].index)

[48, 49, 50, 51, 52, 53, 54, 55]

#### Generate list of `Region`s with their associated "Types" and the Types' "Modifiers"

Note that we will adjust the "Modifier" section for appropriate labeling of fused (or potentially fused) bones. I have submitted a question to clarifie the "Modifier" term on the Slicer forum.

In [6]:
#initialize list for regions and counter for 3dSlicerIntegerLabel

region_list = []
counter = 1
for region in set(df.Region):
    region_idx = list(df.loc[df.Region == region].index)
    #checking if we have a singleton -- Could we have this?
    if len(region_idx) == 1:
         print("Region with no types")
    else:
        region_labels = []
        for i in region_idx:
            #check for paired values & add appropriate Modifiers
            if df.loc[i].Paired == "Y":
                temp_label_dict = {
                    "CodeMeaning": df.loc[i].UberonLabel,
                    "CodingSchemeDesignator": "UBERON",
                    "3dSlicerLabel": df.loc[i].SlicerLabel,
                    "3dSlicerIntegerLabel": counter,                        
                    "CodeValue": str(df.loc[i].UberonID),
                    "contextGroupName": "CommonTissueSegmentationTypes",
                    "paired": "Y",
                    "Modifier": [
                        {
                            "recommendedDisplayRGBValue": [df.loc[i].R, 
                                                            df.loc[i].G, 
                                                            df.loc[i].B],
                            "CodeMeaning": "Right",
                            "CodingSchemeDesignator": "SCT",
                            "3dSlicerLabel": "right" + " " + df.loc[i].SlicerLabel,
                            "3dSlicerIntegerLabel": counter + 1,
                            "cid": "244",
                            "UMLSConceptUID": "C0205090",
                            "CodeValue": "24028007",
                            "contextGroupName": "Laterality",
                            "SNOMEDCTConceptID": "24028007"
                        },
                        {
                            "recommendedDisplayRGBValue": [df.loc[i].R, 
                                                            df.loc[i].G, 
                                                            df.loc[i].B],
                            "CodeMeaning": "Left",
                            "CodingSchemeDesignator": "SCT",
                            "3dSlicerLabel": "left" + " " + df.loc[i].SlicerLabel,
                            "3dSlicerIntegerLabel": counter + 2,
                            "cid": "244",
                            "UMLSConceptUID": "C0205091",
                            "CodeValue": "7771000",
                            "contextGroupName": "Laterality",
                            "SNOMEDCTConceptID": "7771000"
                        },
                        {
                            "recommendedDisplayRGBValue": [df.loc[i].R, 
                                                            df.loc[i].G, 
                                                            df.loc[i].B],
                            "CodeMeaning": "Right and left",
                            "CodingSchemeDesignator": "SCT",
                            "cid": "244",
                            "CodeValue": "0000210",
                            "contextGroupName": "Laterality"
                        }
                        ]
                }
                # increment counter for main + modifiers
                counter += 3
            else:    
                temp_label_dict = {
                    "recommendedDisplayRGBValue": [df.loc[i].R, 
                                                    df.loc[i].G, 
                                                    df.loc[i].B],
                    "CodeMeaning": df.loc[i].UberonLabel, 
                    "CodingSchemeDesignator": "UBERON",
                    "3dSlicerLabel": df.loc[i].SlicerLabel,                       
                    "3dSlicerIntegerLabel": counter,
                    "cid": "7166",
                    "CodeValue": str(df.loc[i].UberonID),
                    "contextGroupName": "CommonTissueSegmentationTypes"
                }
                counter += 1
            #add to list of region's Types
            region_labels.append(temp_label_dict)
            
        #generate region with its corresponding Types
        temp = {"CodeMeaning": region, 
                "CodingSchemeDesignator": "UBERON", 
                "showAnatomy": True,
                "cid": "7150",
                "CodeValue": str(list(df.loc[df.Region == region].UberonID)[0]),  #pull first UberonID?
                "contextGroupName": "SegmentationPropertyCategories",
                "Type": 
                        region_labels
                    }
    #add to list of regions
    region_list.append(temp)

#### Add `region_list` to "Category" of the "SegmentationCodes" for upper level of the file

In [7]:
# combine regions with all their type information

file_test = {
        "SegmentationCategoryTypeContextName": "Segmentation category and type - Test",
        "@schema": "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/segment-context-schema.json#",
        "SegmentationCodes": {
        "Category":
            region_list 
        }
}

Helper function for JSON parsing `int64`. 

Solution adapted from stackexchange: https://stackoverflow.com/a/60376755.

In [8]:
#JSON rejecting int64 type, so we must tell it how to parse
#solution adapted from stackexchange: https://stackoverflow.com/a/60376755

def int_converter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        

Generate a test to check our generated `JSON` input is valid on https://qiicr.org/dcmqi/#/validators (recommended JSON validator from Slicer [Docs](https://slicer.readthedocs.io/en/latest/user_guide/modules/terminologies.html)).

Note: this cell is not required.

In [9]:
#generate test file to check JSON input is valid
#run check on https://qiicr.org/dcmqi/#/validators (recommendation from Slicer Docs)

test = json.dumps(file_test, default = int_converter, indent = 4)
print(test)

{
    "SegmentationCategoryTypeContextName": "Segmentation category and type - Test",
    "@schema": "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/segment-context-schema.json#",
    "SegmentationCodes": {
        "Category": [
            {
                "CodeMeaning": "Hindlimb",
                "CodingSchemeDesignator": "UBERON",
                "showAnatomy": true,
                "cid": "7150",
                "CodeValue": "100056",
                "contextGroupName": "SegmentationPropertyCategories",
                "Type": [
                    {
                        "CodeMeaning": "Hindlimb",
                        "CodingSchemeDesignator": "UBERON",
                        "3dSlicerLabel": "Hindlimb",
                        "3dSlicerIntegerLabel": 1,
                        "CodeValue": "100056",
                        "contextGroupName": "CommonTissueSegmentationTypes",
                        "paired": "Y",
                        "Modifier": [
   

Save test JSON file

In [10]:
with open("generation_test.json", "w", encoding = 'utf-8') as fp:
    json.dump(file_test, fp, default = int_converter, indent = 4)